We will build the real news vs fake news detection engine. We want to demonstrate how this pipeline can be adapted to your organization's specific needs. Instead of using a pre-built dataset, we will download a dataset from Kaggle and utilize it in our fine-tuning process. This approach will help illustrate how the pipeline can be tailored to work with custom datasets in real-world applications.
Here's an outline of the fine-tuning process
1. Import required libraries and packages

2. Load the dataset. Download the data from kaggle and save it on your drive.
3. Load pre-trained BERT tokenizer:


4. Prepare the dataset: 


  * Tokenize the text using the BERT tokenizer
  * Create attention masks
 * Split the dataset into training and validation sets
  * Create a custom PyTorch dataset class (TextClassificationDataset)
  * Instantiate the custom dataset for both training and validation sets
  * Create PyTorch DataLoader
  
4. Load a pre-trained BERT model for sequence classification using the Hugging Face Transformers library
5. Setup Accelarator environment
6. Fine-tune the model:

7. Evaluate the model:
  *Calculate  metrics, such as F1 score, recall, and precision
8. Inference:

  * Create a function to perform inference on new text input
 * Tokenize the input text and convert it to the required format
 * Perform inference using the fine-tuned model
 * Interpret the model's output and return the predicted class

# 1. Import required libraries and packages
 

In [ ]:
import pandas as pd

from sklearn.model_selection import train_test_split


from accelerate import Accelerator

import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

from tqdm import tqdm

from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import AdamW
from transformers import get_scheduler

**Note:** MPS=> Apple's Metal Performance Shaders (MPS) is a framework that provides highly optimized, low-level GPU-accelerated functions for deep learning, image processing, and other compute-intensive tasks.

In [ ]:
def get_device():
  device="cpu"
  if torch.cuda.is_available():
    device="cuda"
  elif  torch.backends.mps.is_available():
    device='mps'
  else:
    device="cpu"
  return device


device = get_device()
print(device) 

mps


# 2. Load Data
1. Reading data from two CSV files: True.csv (real news) and Fake.csv (fake news)
2. Cleaning and preprocessing the data in each CSV file
3. Concatenating both dataframes into a single dataframe
4. The resulting dataframe contains two columns: 'text' for the news content and 'label' for its corresponding category (real or fake)

In [ ]:
real=pd.read_csv('/Users/premtimsina/Documents/bpbbook/chapter4/dataset/True.csv')
fake=pd.read_csv('/Users/premtimsina/Documents/bpbbook/chapter4/dataset/Fake.csv')


In [ ]:
real = real.drop(['title','subject','date'], axis=1)
real['label']=1.0
fake = fake.drop(['title','subject','date'], axis=1)
fake['label']=0.0
dataframe=pd.concat([real, fake], axis=0, ignore_index=True)


In [ ]:
df = dataframe.sample(frac=0.1).reset_index(drop=True)
print(df.head(20))
print(len(df[df['label']==1.0]))
print(len(df[df['label']==0.0]))

                                                 text  label
0   Donald Trump turned a tragedy into an opportun...    0.0
1   MARAWI CITY, Philippines (Reuters) - With a gr...    1.0
2   This could be YUGE news for conservatives who ...    0.0
3   WASHINGTON (Reuters) - The U.S. government is ...    1.0
4   MOGADISHU (Reuters) - More than 300 people wer...    1.0
5   Two white guys living in a state where 96% of ...    0.0
6   WASHINGTON (Reuters) - U.S. Defense Secretary ...    1.0
7                                                        0.0
8   EXETER, N.H./MILFORD, N.H. (Reuters) - U.S. Se...    1.0
9   (Reuters) - New Jersey Governor Chris Christie...    1.0
10  The funniest part of the video comes when MSNB...    0.0
11  Newsweek reporter Kurt Eichenwald came under f...    0.0
12  https://fedup.wpengine.com/wp-content/uploads/...    0.0
13  The Donald Trump campaign is relying heavily o...    0.0
14  BAGHDAD (Reuters) - Arms provided by the Unite...    1.0
15  WASHINGTON (Reuters)

#3.  Load Tokenizer:
1. We are using the `bert-base-uncased` tokenizer. We also need to use the corresponding model

In [ ]:

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# 4. Prepare Data
The data preparation process for BERT-based uncased models involves tokenizing the text, mapping tokens to `input_ids`, creating attention masks `attention_mask`, , and preparing the labels tensor `labels`. Each element of Dataset Class should be dictionary of following structure.

```
{'input_ids': torch.Tensor(),'attention_mask':torch.Tensor(), 'labels': torch.Tensor()  }
```
1. Tokenization: The text input should be tokenized into subwords using BERT's WordPiece tokenizer. This tokenizer converts the text into a format that BERT can understand.

2. `input_ids`: Each token from the tokenized text needs to be mapped to an ID using BERT's vocabulary. The resulting input IDs should be in the form of a tensor or array, usually of shape (batch_size, max_sequence_length).
3. `attention_mask`: The attention mask is used to differentiate between the actual tokens and padding tokens. It has the same shape as the input IDs tensor, i.e., (batch_size, max_sequence_length). The mask has 1s for actual tokens and 0s for padding tokens.
4. `labels`: The labels tensor contains the true class or value for each example in the dataset. It usually has a shape of (batch_size,). For classification tasks, these labels are one-hot-encoded labels

In [ ]:
# this is just creating list of tuples. Each tupe has (text, label)
data=list(zip(df['text'].tolist(), df['label'].tolist()))

# This function takes list of Texts, and Labels as Parameter
# This function return input_ids, attention_mask, and labels_out
def tokenize_and_encode(texts, labels):
    input_ids, attention_masks, labels_out = [], [], []
    for text, label in zip(texts, labels):
        encoded = tokenizer.encode_plus(text, max_length=512, padding='max_length', truncation=True)
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
        labels_out.append(label)
    return torch.tensor(input_ids), torch.tensor(attention_masks), torch.tensor(labels_out)

# seprate the tuples
# generate two lists: a) containing texts, b) containing labels
texts, labels = zip(*data)

# train, validation split
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2)

# tokenization
train_input_ids, train_attention_masks, train_labels = tokenize_and_encode(train_texts, train_labels)
val_input_ids, val_attention_masks, val_labels = tokenize_and_encode(val_texts, val_labels)




**It's always good to review the data**
1. input_ids
  * `0` token value means padded token
2. attention_mask
  * `1`: corresponding token is real token
  * `0`: corresponding token is padded token

In [ ]:
print('train_input_ids ',train_input_ids[0].shape ,train_input_ids[0], '\n'
      'train_attention_masks ', train_attention_masks[0] ,train_attention_masks[0], '\n'
      'train_labels', train_labels[0])

train_input_ids  torch.Size([512]) tensor([  101,   153,  9919, 11612, 11840,  2069,   117,  3658,   113, 11336,
        27603,   114,   118,  1109, 19850,  9651, 17677,  1372,  1163,  1122,
         2446,  1149,  1126,  2035,  1113,  3625,  1115,  1841,  1421,  1234,
         1107,  1103, 10059,  1877, 20268,  6469,   119, 12008,  8167, 20059,
          118,   174,   118, 17677,  3658,   113,   157, 17433,   114, 15465,
        12460,   148, 26033, 11192,  7192,  1163,  1103, 19342,  8100,  1126,
        22421, 12154,  4442,  1106,  4010,  2699,  4675,  1107,  1103, 21519,
         2149,  5571,  1298,  1115,  1110,  1226,  1104,  1103,  3467,  1193,
        24930, 25685, 25924, 24252, 20371,   113,  6820,  9159,   114,   119,
         1109,  9232,  4168,   170,  3686,  1107,  1103,  7085, 13889,  1298,
         1104, 21519,  2149,   117,  3646,  1300,  2699,  4675,  1105,  1126,
         2078,  1121,  1103,  6688,  3469,   117,  1469,  1433,  3509,  1163,
          119, 21519,  2149, 

### TextClassificationDataset
1. For tunning `bert-based-uncased`: each item of Dataset must be of type dictionary with at following  keys:
  * input_ids
  * attention_mask
  * labels
2. Thus,  `__getitem__`  should return dictionary of following structure:
```
{
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_masks[idx],
            'labels': self.one_hot_labels[idx]
        }
```
3. one_hot_encode method: A static method that takes in targets (labels) and num_classes as arguments. It converts the given targets into one-hot encoded tensors. The method first converts the targets to long tensors and then initializes a zero tensor of shape (number of samples, num_classes). The scatter_ function is used to place 1.0 in the appropriate position for each sample's label, resulting in a one-hot encoded tensor.

In [ ]:
class TextClassificationDataset(torch.utils.data.Dataset):
    def __init__(self, input_ids, attention_masks, labels, num_classes=2):
        self.input_ids = input_ids
        self.attention_masks = attention_masks
        self.labels = labels
        self.num_classes = num_classes
        self.one_hot_labels = self.one_hot_encode(labels, num_classes)

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_masks[idx],
            'labels': self.one_hot_labels[idx]
        }


    @staticmethod
    def one_hot_encode(targets, num_classes):
        targets = targets.long()
        one_hot_targets = torch.zeros(targets.size(0), num_classes)
        one_hot_targets.scatter_(1, targets.unsqueeze(1), 1.0)
        return one_hot_targets
        

train_dataset = TextClassificationDataset(train_input_ids, train_attention_masks, train_labels)
val_dataset = TextClassificationDataset(val_input_ids, val_attention_masks, val_labels)


### DataLoader
*italicized text*

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
eval_dataloader = DataLoader(val_dataset, batch_size=8)

In [ ]:
print(len(train_dataset))
len((val_dataset))

3592


898

1.Revisiting dimension requirements for Transformers in Pytorch from Chapter 3: The encoder expects data with dimensions (seq_len, batch_size). However, Hugging Face's bert-based-uncased model requires data with dimensions (batch_size, seq_len). As a result, the output from the train_dataloader has dimensions of (batch_size, seq_len).

In [ ]:
item=next(iter(train_dataloader))
item_ids,item_mask,item_labels=item['input_ids'],item['attention_mask'],item['labels']
print ('item_ids, ',item_ids.shape, '\n',
       'item_mask, ',item_mask.shape, '\n',
       'item_labels, ',item_labels.shape, '\n',)

item_ids,  torch.Size([8, 512]) 
 item_mask,  torch.Size([8, 512]) 
 item_labels,  torch.Size([8, 2]) 



In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
optimizer = AdamW(model.parameters(), lr=5e-5)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

# 5. Prepare Accelaerator
What is Accelerator?
 1. It provides an easy-to-use API for training deep learning models on various hardware accelerators, such as GPUs, TPUs, and Apple's Metal Performance Shaders (MPS).
  * In our example, during training, we donot specifically select 'mps' device. THe accelerator automatically detects it and use 'mps' for training
 2. The Accelerator library is particularly useful for distributed training and mixed-precision training.

In [ ]:
# Prepare the model and optimizer
accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)


# 5. Fine Tune The Model
1. `lr_scheduler` in the provided code is an instance of a learning rate scheduler, which is responsible for adjusting the learning rate during the training process. The learning rate scheduler helps improve the training process by dynamically adjusting the learning rate based on the number of training steps. In this code, the learning rate starts with the initial value set in the optimizer and decreases linearly to 0 as the training progresses.
2. Some benefit of lr_scheduler over optimizer alone are
  * Faster convergence 
  * Avoid Overshooting: When using a fixed learning rate, the optimizer might overshoot the optimal solution, especially in the later stages of training. By decreasing the learning rate over time, the model can make smaller updates and fine-tune its weights
  
3. `progress_bar` is just utility to show the progress of training
4. These are standard approach for fine tunning:
```
 }
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
```
  * each batch should be dictionary of structure {input_ids:torch.Tensor(), attention_mask: torch.Tensor(), labels: torch.Tensor()
  * the dimension of input_ids=(batch_size, seq_len); attention_mask= (batch_size, seq_len); and labels=(batch_size,)
  * You can notice that during training, we are not explicitly converting `tensor` into device; accelerator is automatically identifying the `device` and converting `tensor` into the appropriate format
1. After each epoch, we are also printing the evaluation metrics over the evaluation dataset

In [ ]:

num_epochs = 1
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
      "linear",
      optimizer=optimizer,
      num_warmup_steps=0,
      num_training_steps=num_training_steps
  )
progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_epochs):
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
    model.eval()
    device = 'mps'
    preds = []
    out_label_ids = []
    epochs=1
    epoch=1

    for batch in eval_dataloader:
        with torch.no_grad():
            inputs = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**inputs)
            logits = outputs.logits

        preds.extend(torch.argmax(logits.detach().cpu(), dim=1).numpy())
        out_label_ids.extend(torch.argmax(inputs["labels"].detach().cpu(),dim=1).numpy())
    accuracy = accuracy_score(out_label_ids, preds)
    f1 = f1_score(out_label_ids, preds, average='weighted')
    recall = recall_score(out_label_ids, preds, average='weighted')
    precision = precision_score(out_label_ids, preds, average='weighted')

    print(f"Epoch {epoch + 1}/{num_epochs} Evaluation Results:")
    print(f"Accuracy: {accuracy}")
    print(f"F1 Score: {f1}")
    print(f"Recall: {recall}")
    print(f"Precision: {precision}")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 449/449 [04:19<00:00,  1.71it/s]

Epoch 2/1 Evaluation Results:
Accuracy: 0.9977728285077951
F1 Score: 0.9977724507291988
Recall: 0.9977728285077951
Precision: 0.9977820316957794


# 6. Inference Pipeline
1. `tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
`: You need use the same tokenizer that was use for fine-tunning
2. `logits.detach().cpu()`
  * `detach is done to prevent  unintentional back-propogation
  * `.cpu` is done so that the output is compatible with scikit-learn libraries for further computation

In [ ]:
from transformers import BertTokenizer
import torch
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def inference(text, model,  label, device='mps'):
    # Load the tokenizer

    # Tokenize the input text
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    # Move input tensors to the specified device (default: 'cpu')
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Set the model to evaluation mode and perform inference
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    # Get the index of the predicted label
    pred_label_idx = torch.argmax(logits.detach().cpu(), dim=1).item()

    print(f"Predicted label index: {pred_label_idx}, actual label {label}")
    return pred_label_idx


In [ ]:
#https://abcnews.go.com/US/tornado-confirmed-delaware-powerful-storm-moves-east/story?id=98293454
text='\
WASHINGTON (ABC) A confirmed tornado was located near Bridgeville in Sussex County, Delaware, shortly after 6 p.m. ET Saturday, moving east at 50 mph, according to the National Weather Service. Downed trees and wires were reported in the area.\
'
inference(text, model, 1.0)
text="this is definately junk text I am typing"
inference(text, model, 0.0)

Predicted label index: 1, actual label 1.0
Predicted label index: 0, actual label 0.0


0